# COGS 118B - Final Project

# Names

- Sam Dimmock
- Vincent Gao
- Owen Guan
- Ryota Takemura


# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- what your goal/problem is
- what the data used represents 
- the solution/what you did
- major results you came up with (mention how results are measured) 

__NB:__ this final project form is much more report-like than the proposal and the checkpoint. Think in terms of writing a paper with bits of code in the middle to make the plots/tables

# Abstract (130 words)
Is it possible to design an AI algorithm that behaves similar to human players ranging from beginning to master for the simple-styled Shanghai mahjong? The data used represents training data by simulations and measures the skill expression of our agent given a starting hand. Our environment is used to guide the AI algorithm how to behave as an agent with a goal to win but with differentiating skill levels. The performance level is measured by playing alone, against other players and against oracle agents to reach the intended level of skill. Though the original standard for the AI algorithm was not met, the process of designing a purposeful mahjong environment, reward prediction algorithm, functionable agent, and monte carlo algorithm allowed for discoveries of efficiencies, and reasonable simplifications for runtime/feasibility.

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 3 to 5 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts. 

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated. 

Mahjong is an iconic multiplayer imperfect information game that has been played for nearly 2 centuries around the world, originating in China in the mid-1800s<a name="lang"></a>[<sup>[1]</sup>](#langnote). One of the reasons it is so beloved by millions of players is because of the flexible rulesets, point system, complex strategies, and fun it brings to all players. However, it is a game involving very much stochasticity as each of the players try to find a winning set of 14 tiles. While there are certainly many mahjong AI algorithms like those used in online games such as Mahjong Soul and Microsoft’s Suphx, the goal is to create an optimized AI algorithm for Shanghai mahjong that is fair and behaves in a manner similar to humans.


The best way to analyze training data is simply by using a trainer like those created by players such as Euophrys<a name="euophrys"></a>[<sup>[2]</sup>](#euophrysnote) to scrape actual gameplay from online records (such as Mahjong Soul’s database) for different levels of play; this helps for understanding different methods of playing mahjong as well as different rulesets (Mahjong Soul’s style is called jantama/riichi mahjong). On the other hand, when designing an optimal algorithm, it is important to look into Suphx and the way deep reinforcement learning is used to formulate the best strategy.<a name="li"></a>[<sup>[3]</sup>](#linote) The design approach was decided with techniques such as run-time policy adaptation, global reward prediction, and oracle guiding to reach the top 0.1% in terms of ranking on the Tenhou gaming service.<a name="li"></a>[<sup>[3]</sup>](#linote) Due to the nature of the game having up to 53 tiles of the total 136 tiles being hidden at any given time.it is difficult to build a standard game tree so certain techniques such as Monte-Carlo tree search and search min-max algorithms have limited applications. The way Suphx circumvents these difficulties for jantama is through policy gradient algorithms, global reward prediction of a final reward of hypothetical future rounds via learning signals, agent based training on an oracle agent with perfect information that slowly decreases access until it becomes a normal agent, and finally, a parametric Monte-Carlo policy adaptation to improve run-time.<a name="li"></a>[<sup>[3]</sup>](#linote) It is important to keep in mind that the difficulties experienced when designing an implementation for an optimal riichi mahjong AI may not be the exact same for a simplistic Shanghai mahjong ruleset, especially if the AI is more imitation/difficulty based in design.

The core framework of mahjong can be compartmentalized to four key considerations that can be used for structuring an AI: mahjong is a multi-agent four player game with no teams; mahjong is imperfect information so nothing about the hidden tiles or opponents’ tiles are directly known; not including the 13 starting tiles that are known, the initiate possible states, there are nearly 5 x 10128 possible hidden states in the set of all states, forming a hidden Markov model for each player; and finally, the cost-reward application for mahjong ranges completely differently depending on the game state, as a good hand very much depends on the hidden markov model in order to determine possible winning hands.<a name="koyamada"></a>[<sup>[4]</sup>](#koyamadanote) This framework can be applied to both riichi and Shanghai mahjong and will serve as the fundamentals for how training data is approached and how deep reinforcement learning will be used for this particular system.

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

The problem of playing Mahjong can be split into two main sub-problems: deciding what tiles to pick up (that is, whether to pong or chow if possible) and deciding which tile to discard. A fully-developed agent will have learned how to do both, but for the purposes of development we found it best to train an agent on only one at a time. Of the two, the decision of what tiles to pick up can be made easily using simple, relatively effective heuristics derived from domain knowledge, so we decided to train an agent to decide which tiles to discard.

# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

Ways of encoding state space:
* Lists of hand and discard piles for each player, containing Tile objects (mahjong.py)
    * Easiest to implement game logic
* Vector encoding: position = tile type, value = number of tiles (agent.py)
    * Vector for each player's hand and for the discard pile, combined in matrix or flattened matrix
    * Ideal input for neural network
    * Should be possible to implement full game logic using this encoding, which would likely make things faster, but have not done so
* Sam's encoding - non-functional, probably shouldn't be discussed in any real detail

Ways of encoding action space:
* 

Approaches to training agent:
* Monte Carlo value estimates (reward_prediction.py)
* Monte Carlo control
* Oracle agent (agent.py) - supplemental method, can be used to accelerate learning (oracle agent still has to be trained)

Implementation details:
* All agents implemented in PyTorch
    * Currently Ryota's only has the model implemented in PyTorch, but this should be easily changed
    * Advantage of having agent implemented in PyTorch, not just model: can shift entire agent to `device`, minimizing accidental cross-device computations (which raise errors)
* Ideal implementation for vector encoding also in PyTorch to reduce computational overhead from creating new tensors
* Implementation of Monte Carlo control based on implementation learned in class
* Implementation of oracle agent based on article [I forgot which article this is]
* Implementation of game mechanics, heuristics:
    * We have multiple somewhat different implementations for this, and should probably at least try to standardize them for the sake of presenting a consistent narrative and making it seem like one group project instead of multiple individual projects.
* 

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

# Results

You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Probably you should include a learning curve to demonstrate how much better the model gets as you increase the number of trials

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Generally reinforement learning tasks may require a huge amount of training, so extensive grid search is unlikely to be possible. However expoloring a few reasonable hyper-parameters may still be possible.  Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?  Or you compare a completely different approach/alogirhtm to the problem? Whatever, this stuff is just serving suggestions.



# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.


### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   


### Future work
Looking at the limitations and/or the toughest parts of the problem and/or the situations where the algorithm(s) did the worst... is there something you'd like to try to make these better.

### Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination.

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

# Footnotes
<a name="langnote"></a>1.[^](#lang) Lang, C. (2021, May 4). What the surprising history of Mah-jongg can teach us about America. TIME. https://time.com/6045817/mahjongg-history/

<a name="euophrysnote"></a>2.[^](#euophrys) Euophrys. (n.d.). GitHub - Euophrys/Riichi-Trainer: A Riichi Mahjong trainer. GitHub. https://github.com/Euophrys/Riichi-Trainer

<a name="linote"></a>3.[^](#li) Li, J., Koyamada, S., Ye, Q., Liu, G., Wang, C., Yang, R., Zhao, L., Qin, T., Liu, T., & Hon, H. (2021, January 25). Suphx: Mastering Mahjong with Deep Reinforcement Learning - Microsoft Research. Microsoft Research. https://www.microsoft.com/en-us/research/publication/suphx-mastering-mahjong-with-deep-reinforcement-learning/

<a name="koyamadanote"></a>2.[^](#koyamada) Koyamada, S., Habara, K., Goto, N., Okano, S., Nishimori, S., & Ishii, S. (n.d.). Mjx: A framework for Mahjong AI research. 2021 IEEE Conference on Games (CoG), 504–507. https://doi.org/10.1109/cog51982.2022.9893712